In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

In [4]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1127141, 6443175, 958729, 17104566, 1108854]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1127141, 6443175, 958729, 17104566, 1108854]"


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

In [5]:
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2282320,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0
2282321,462,41297773713,635,993339,1,1.99,304,0.00,2040,91,0.0,0.0
2282322,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0


In [6]:
%%time
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
# товар-пользователь
#  на входе: список товаров и пользователи, которым они подходят
model.fit(sparse_user_item.T)  # На вход item-user matrix
#          show_progress=True)


result['itemitem_without999999'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine_without999999'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf_without999999'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases_without999999'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])


CPU times: user 1min 53s, sys: 4.66 s, total: 1min 58s
Wall time: 1min 7s


##### Код для получения результатов приведен ниже
##### Результаты:
###### Без 999999:
* Для itemitem_without999999 precision = 15.409999999999998
* Для cosine_without999999 precision = 12.959999999999999
* Для tfidf_without999999 precision = 12.29
* Для own_purchases_without999999 precision = 21.91

###### C 999999:
* Для itemitem precision = 13.69
* Для cosine precision = 13.29
* Для tfidf precision = 13.900000000000002
* Для own_purchases precision = 17.97

##### Качество падает ненамного, а скорость - в разы увеличивается при использовании 99999

In [7]:
%%time
#### Вернем то, что было на лекции
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
# товар-пользователь
#  на входе: список товаров и пользователи, которым они подходят
model.fit(sparse_user_item.T)  # На вход item-user matrix
#          show_progress=True)


result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

/home/andreyl/Soft/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)



CPU times: user 12.1 s, sys: 183 ms, total: 12.3 s
Wall time: 6.09 s


In [8]:
result.head(2)

,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem_without999999,cosine_without999999,tfidf_without999999,own_purchases_without999999,itemitem,cosine,tfidf,own_purchases
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1127141, 6443175, 958729, 17104566, 1108854]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1127141, 6443175, 958729, 17104566, 1108854]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
import random
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    data_to_weight = items_weights.groupby('item_id')['sales_value'].sum()
    data_to_weight = data_to_weight[data_to_weight>0]
    popular_weighted = pd.DataFrame(np.log10(data_to_weight))
    result = []
    kmax=n
#     step=1
    while len(set(result))<n:
        result = random.choices(popular_weighted.index,weights=popular_weighted.values,k=kmax)
        kmax+=kmax
#         step+=1
#     print('Сделано шагов:',step)
    return list(set(result))

Сделайте предсказания

In [10]:
%%time
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
popular_weighted = weighted_random_recommendation(data_train, n=5)
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: popular_weighted)
result.head(2)

CPU times: user 286 ms, sys: 3.99 ms, total: 290 ms
Wall time: 289 ms


,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem_without999999,cosine_without999999,tfidf_without999999,own_purchases_without999999,itemitem,cosine,tfidf,own_purchases
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [11]:
# your_code
def precision_at_k(recommended_list, bought_list, k=10):
#     print(recommended_list)
    bought_list = np.array([int(z) for z in bought_list[1:-1].split()])
    try:
        recommended_list = np.array([int(z) for z in recommended_list[1:-1].split(', ')])
    except Exception:
        pass
#     print(recommended_list[0])
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
#     print(bought_list)
    flags = np.isin(recommended_list,bought_list)
#     print(flags)

    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

for i in result.columns[3:]:
    precision = 0
    for ii in range(result.shape[0]):
        precision+=precision_at_k(result[i][ii], result['actual'][ii], k=5)
    print('Для',i,'precision =',round(precision/len(result),4)*100)

Для random_recommendation precision = 0.08
Для popular_recommendation precision = 15.52
Для weighted_random_recommendation precision = 0.0
Для itemitem_without999999 precision = 15.409999999999998
Для cosine_without999999 precision = 12.959999999999999
Для tfidf_without999999 precision = 12.29
Для own_purchases_without999999 precision = 21.91
Для itemitem precision = 13.69
Для cosine precision = 13.29
Для tfidf precision = 13.900000000000002
Для own_purchases precision = 17.97


##### Если не рассматривать случаи без использования товара 999999 (которые очень долго выполняются при большом объеме данных могут быть просто неэффективными), то:
##### Неплохое качество показывает алгоритм **popular recommendation**; однако, надо учитывать, что он показывает популярные товары, которые могут и не быть продуктом, который требуется продавать чаще.
##### Наиболее интересны алгоритмы **itemitem** и **own_purchases precision**, в которых учитываются не только частота покупок другими пользователями, но и похожесть этих пользователей на анализируемого пользователя


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

### 1.1 Random recommendation

In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

items = data_train.item_id.value_counts()
items = items.index[:5000].values

result['random_rec_improved'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 319 ms, sys: 984 µs, total: 320 ms
Wall time: 323 ms


,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem_without999999,cosine_without999999,tfidf_without999999,own_purchases_without999999,itemitem,cosine,tfidf,own_purchases,random_rec_improved
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1043494, 10204333, 883916, 859154, 916381]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1136719, 1126890, 5565369, 12946027, 901976]"


### 1.2 Popularity-based recommendation

In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
%%time
data_train_improved = data_train.loc[data_train['item_id'].isin(items),:]
# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_improved, n=5)

result['popular_rec_improved'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 433 ms, sys: 2.84 ms, total: 436 ms
Wall time: 435 ms


,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem_without999999,cosine_without999999,tfidf_without999999,own_purchases_without999999,itemitem,cosine,tfidf,own_purchases,random_rec_improved,popular_rec_improved
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1043494, 10204333, 883916, 859154, 916381]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1136719, 1126890, 5565369, 12946027, 901976]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [16]:
import random
def weighted_random_recommendation(data_to_weight, n=5):
    data_to_weight = data_to_weight.groupby('item_id')['sales_value'].sum()
    data_to_weight = data_to_weight[data_to_weight>0]
    popular_weighted = pd.DataFrame(np.log10(data_to_weight))
    result = []
    kmax=n
    step=1
    while len(set(result))<n:
        result = random.choices(popular_weighted.index,weights=popular_weighted.values,k=kmax)
        kmax+=kmax
        step+=1
    print('Сделано шагов:',step)
    return list(set(result))

In [17]:
%%time

popular_weighted = weighted_random_recommendation(data_train_improved, n=5)
result['weighted_random_rec_improved'] = result['user_id'].apply(lambda x: popular_weighted)
result.head(2)

Сделано шагов: 2
CPU times: user 115 ms, sys: 1.06 ms, total: 116 ms
Wall time: 115 ms


,Unnamed: 0,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem_without999999,cosine_without999999,tfidf_without999999,own_purchases_without999999,itemitem,cosine,tfidf,own_purchases,random_rec_improved,popular_rec_improved,weighted_random_rec_improved
0,0,1,[ 821867 834484 856942 865456 889248 ...,"[9336100, 8156313, 1318508, 1017772, 1087199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1043494, 10204333, 883916, 859154, 916381]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1008547, 882595, 956486, 1093966, 844759]"
1,1,3,[ 835476 851057 872021 878302 879948 ...,"[12385978, 202364, 9884686, 857163, 1906461]","[6534178, 6533889, 1029743, 6534166, 1082185]","[710659, 950200, 12605657, 13777086, 1014751]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1136719, 1126890, 5565369, 12946027, 901976]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1008547, 882595, 956486, 1093966, 844759]"


In [ ]:
model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

In [21]:
model = ItemItemRecommender(K=10, num_threads=12) # K - кол-во билжайших соседей
# товар-пользователь
#  на входе: список товаров и пользователи, которым они подходят
model.fit(sparse_user_item.T)  # На вход item-user matrix
#          show_progress=True)


result['itemitem_10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=8, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


model = ItemItemRecommender(K=20, num_threads=12) # K - кол-во билжайших соседей
# товар-пользователь
#  на входе: список товаров и пользователи, которым они подходят
model.fit(sparse_user_item.T)  # На вход item-user matrix
#          show_progress=True)


result['itemitem_20'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=8, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [22]:
# your_code
def precision_at_k(recommended_list, bought_list, k=10):
#     print(recommended_list)
    bought_list = np.array([int(z) for z in bought_list[1:-1].split()])
    try:
        recommended_list = np.array([int(z) for z in recommended_list[1:-1].split(', ')])
    except Exception:
        pass
#     print(recommended_list[0])
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
#     print(bought_list)
    flags = np.isin(recommended_list,bought_list)
#     print(flags)

    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

for i in result.columns[3:]:
    precision = 0
    for ii in range(result.shape[0]):
        precision+=precision_at_k(result[i][ii], result['actual'][ii], k=5)
    print('Для',i,'precision =',round(precision/len(result),4)*100)

Для random_recommendation precision = 0.08
Для popular_recommendation precision = 15.52
Для weighted_random_recommendation precision = 0.0
Для itemitem_without999999 precision = 15.409999999999998
Для cosine_without999999 precision = 12.959999999999999
Для tfidf_without999999 precision = 12.29
Для own_purchases_without999999 precision = 21.91
Для itemitem precision = 13.69
Для cosine precision = 13.29
Для tfidf precision = 13.900000000000002
Для own_purchases precision = 17.97
Для random_rec_improved precision = 0.5700000000000001
Для popular_rec_improved precision = 15.52
Для weighted_random_rec_improved precision = 0.6
Для itemitem_10 precision = 15.09
Для itemitem_20 precision = 15.260000000000002


##### Для popular_rec_improved результат не улучшился - это ожидаемо, т.к. сам алгоритм основан на выборе из наиболее популярных товаров;
##### Что касается random'ного и взвешенного рандомного, то они значительно улучшились.
##### Алгоритм itemitem улучшается при увеличении кол-ва соседей; вероятно, это имеет некий предел, т.к. при переходе от 5 к 10 качество улучшается на 3%, а при переходе от 10 к 20 - всего на 0,2%. Можно осуществить подбор лучшего параметра графически.

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

### Считаем матрицу синусного подобия (similarity)

In [8]:
from numpy import array as a
from numpy.random import random_integers as randi
from numpy.linalg.linalg import norm
from numpy import set_printoptions

m_dot_m = user_item_matrix.dot(user_item_matrix.T);       
norm = a([norm(user_item_matrix, axis=1)]) * a([norm(user_item_matrix, axis=1)]).T; 
cossim = m_dot_m / norm
cos_result = pd.DataFrame(columns=['user','user1','user2','user3','user4','user5'])
z=0
for i in cossim.columns:
    cos_result.loc[z,['user1','user2','user3','user4','user5']]=cossim.loc[i,:].nlargest(5).index.values
    cos_result.loc[z,'user']=i
    z+=1
cos_result

,user,user1,user2,user3,user4,user5
0,1,1,2171,1726,1510,1424
1,2,2,19,2020,198,68
2,3,3,2412,1024,1752,1883
3,4,4,662,612,1039,1678
4,5,5,1830,2138,1978,1230
...,...,...,...,...,...,...
2494,2496,2496,306,591,1024,361
2495,2497,2497,1396,647,1024,1430
2496,2498,2498,1598,2284,1720,1367
2497,2499,2499,253,1024,878,1489


### Добавляем оценку для товаров каждому пользователю: оценка = факт покупки (1 или 0) * количество проданных товаров
### "Факт покупки" используется в связи с тем, что клиент мог купить товар несколько раз; тогда при умножении получится "двойной зачет", хотя все это зависит от логики, с которой хранились данные
### Цифры получаются очень большие, поэтому приводятся к среднему (от -1 до 1)

In [9]:
summa = 0
cos_result['score']=''
tops = popularity.sort_values('n_sold', ascending=False).set_index('item_id')
for z in cos_result['user']:
    try:
        for i in user_item_matrix.loc[z,:][user_item_matrix.loc[z,:]>0].index.values:
            try:
                summa+=i*tops.loc[i,'n_sold']
            except Exception:
                pass
        cos_result.loc[cos_result['user']==z,'score'] = summa
        summa=0
    except Exception:
        pass
cos_result['score'] = (cos_result['score']-cos_result['score'].mean()) / (cos_result['score'].max()-cos_result['score'].min())
cos_result
# sim - "похожесть пользователей"
# r - их "оценка", основанная на количестве покупок (точнее, их разнообразии) и стоимости совершенных всеми пользователями аналогичных покупок
# каждый товар "похожих" пользователей получит вес от них в виде r
# mu - средняя оценка похожих пользователей; 

,user,user1,user2,user3,user4,user5,score
0,1,1,2171,1726,1510,1424,-0.443416
1,2,2,19,2020,198,68,-0.443323
2,3,3,2412,1024,1752,1883,0.41657
3,4,4,662,612,1039,1678,-0.443544
4,5,5,1830,2138,1978,1230,-0.443593
...,...,...,...,...,...,...,...
2494,2496,2496,306,591,1024,361,0.416565
2495,2497,2497,1396,647,1024,1430,0.416686
2496,2498,2498,1598,2284,1720,1367,-0.371118
2497,2499,2499,253,1024,878,1489,0.488742


### Считаем среднюю "похожесть" для всех пользователей, которые близки ползователю в столбце 'user'

In [10]:
for i in cos_result['user']:
    mu = cos_result.loc[cos_result['user']==i,['user1','user2','user3','user4','user5']].values[0]
    mu = cossim.loc[i,mu].mean()
    cos_result.loc[cos_result['user']==i,'mu']=mu
cos_result

,user,user1,user2,user3,user4,user5,score,mu
0,1,1,2171,1726,1510,1424,-0.443416,0.360479
1,2,2,19,2020,198,68,-0.443323,0.368940
2,3,3,2412,1024,1752,1883,0.41657,0.430564
3,4,4,662,612,1039,1678,-0.443544,0.332027
4,5,5,1830,2138,1978,1230,-0.443593,0.303542
...,...,...,...,...,...,...,...,...
2494,2496,2496,306,591,1024,361,0.416565,0.405681
2495,2497,2497,1396,647,1024,1430,0.416686,0.430962
2496,2498,2498,1598,2284,1720,1367,-0.371118,0.373850
2497,2499,2499,253,1024,878,1489,0.488742,0.424789


### Неизвестно, что за параметр sales_value, но я решил, что это некая сумма (в рублях или тысячах рублей), которую можно использовать в качестве множителя (чем больше сумма, тем важнее для нас продавать и рекомендовать этот товар)

In [11]:
popularity_and_sale = data_train.groupby('item_id')[['quantity','sales_value']].sum().reset_index()
popularity_and_sale['sales_value'] = (popularity_and_sale['sales_value']-popularity_and_sale['sales_value'].mean())/(popularity_and_sale['sales_value'].max()-popularity_and_sale['sales_value'].min())
popularity_and_sale

,item_id,quantity,sales_value
0,202291,35911,-0.000443
1,397896,1214994,0.000484
2,420647,168661,-0.000316
3,480014,371107,-0.000182
4,545926,20134,-0.000451
...,...,...,...
4996,15926886,260,-0.000275
4997,15927403,376,-0.000396
4998,15927661,230,-0.000451
4999,15927850,269,-0.000416


### В итоге, рассчитываем и выводим список рекомендованных 5 товаров;

In [22]:
%%time
cos_result['recommendations']=''
cos_result['recommendations']=cos_result['recommendations'].astype('object')
allitems = user_item_matrix[user_item_matrix.index==1].columns
for k,z in enumerate(cos_result['user']):
    user_list = cos_result.loc[cos_result['user']==z,['user1','user2','user3','user4','user5']].values[0]
#     print(k,z)
    nonzero_items = np.array([i for i in allitems if user_item_matrix.loc[user_item_matrix.index==z,i].values>0])
    for user in user_list:
        mu,r,rmean,sim=0,0,0,0
        mu = cos_result.loc[cos_result['user']==user,'mu']
        r = cos_result.loc[cos_result['user']==user,'score']
        rmean = cos_result.loc[cos_result['user'].isin(user_list),'score'].mean()
        sim = cossim[z].loc[user]
        r_total = mu.values[0]+rmean + sim * (r.values[0] - rmean - mu.values[0])
    recs = [i1 for i1,i2 in sorted(((i,abs(r_total*popularity_and_sale.loc[popularity_and_sale['item_id']==i,'sales_value'].mean())) for i in nonzero_items), key=lambda x: x[1])[::-1][:5]]
#     print(recs)
    cos_result.at[k,'recommendations']=recs
cos_result.head()

CPU times: user 43min 47s, sys: 3.58 s, total: 43min 51s
Wall time: 43min 57s


,user,user1,user2,user3,user4,user5,score,mu,recommendations
0,1,1,2171,1726,1510,1424,-0.443416,0.360479,"[999999, 1029743, 1082185, 995242, 1005186]"
1,2,2,19,2020,198,68,-0.443323,0.368940,"[999999, 1082185, 916122, 1106523, 5569230]"
2,3,3,2412,1024,1752,1883,0.41657,0.430564,"[999999, 6534178, 1082185, 1106523, 5569230]"
3,4,4,662,612,1039,1678,-0.443544,0.332027,"[999999, 1029743, 5569230, 1044078, 1075368]"
4,5,5,1830,2138,1978,1230,-0.443593,0.303542,"[999999, 1029743, 916122, 874972, 1126899]"


In [24]:
precision = 0
def precision_at_k(recommended_list, bought_list, k=10):
    bought_list = np.array([int(z) for z in bought_list[1:-1].split()])
    try:
        recommended_list = np.array([int(z) for z in recommended_list])
    except Exception:
        pass
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list,recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision


for ii in range(result.shape[0]):
    a=precision_at_k(cos_result['recommendations'][ii], result['actual'][ii], k=5)
    if a>=0:
        precision+=a
        
print('precision =',round(precision/len(result),4)*100)



precision = 13.16


##### Получаемая точность - явно не лучшая, среди остальных алгоритмов; однако плюсом в данном случае является использование множителя, отражающего стоимость товара; возможно, благодаря этому, доход от использования этого алгоритма будет лучше, чем от других.